In [1]:
import numpy as np
import matplotlib.pyplot as plt

# Notes on 
[Р.В. Шамин. Лекция 3. Генетические алгоритмы в задачах экономики и менеджмента](http://www.mathnet.ru/php/seminars.phtml?option_lang=rus&presentid=19513)

- [X] Формальное определение генетического алгоритма
- [X] Схема алгоритма
- [X] Прикладные задачи  
    - [X] Модификация и переложение алгоритма для решение прикладных задач менеджмента
- [X] Пример

## Формальное определение генетического алгоритма

#### Оптимизационная задача в общем смысле

Пусть $U$ - множество возможных рещений, $|U| > 1$, 
$U$ не пусто и  содержит более одного элемента.

Задана действитейльная функция $f:\mapsto\mathbb{R}$

Смысл задачи в нахождении максимума функции.

#### Формальное определение генетического алгоритма

$f(x) \to max, x\in U$

$X = (X_1, X_2, ...,X_N), X_i\in \mathbb{R}, X_i\in U_i\subset \mathbb{R}$

$X$ - ген, $X_i$ - хромосома

$Fit(x)$ - Фитнес функция. $Fit(x)\to max$

Наша цель найти ген оптимизирующий заданную фитнес функцию.

1. Порождение нового гена
2. Скрещивание $(X,Y)\mapsto Z$  
    $Z=(Z_1,Z_2, ...,Z_N)$  
    $Z_i \in \{X_i,Y_i\}$
3. Мутация $X\to \overline{X}, \overline{X} - близка X$

## Схема алгоритма

1. Создаем пул генов $(X^1,X^2, ...,X^M)$
2. Сортируем пул по значению фитнес функции $Fit_i=Fit(X^i)$   
    $Fit_1\geqslant Fit_2\geqslant ...\geqslant Fit_M$
3. Пусть $k\in (0,M)$, количество особей(генов) на обновление. Например 20%.  
    $X^i:=$ Создаем новый ген $i=M-k+1,M-k+2, ...,M$
4. Скрещивание генов из пула  
    - $X^1$ сохраняем(в таком случае получаем монотонный алгоритм, в некоторых случаях есть смысл менять и "царя горы"
    - $X^i:=$ Скрещиваем $X^m$ и $X^k$, выбирая $m,k$ с вероятностями пропорциональными их нормализованной фитнес функции
    - Переходим в 2. пока не получим удовлетворяющее нас условие

## Прикладные задачи 

$F(x) \to max, x\in U$

* Оптимизационные задачи, связанные со структурой экономического объекта
* Оптимизация планирования(формального, неформального)
* Позволяют за несколько итераций понять сильные и слыбые стороны плана
* Решение кадровых вопросов
    - Алгоритм привязываеться к реальным показателям сотрудников/потенциальных членов команды
    - Позволяет решить задачу оптимизации сотрудников внутри команды за минимальное количество итераций
    - В качестве фитнес функции может выступать результат "деловой игры" или "бизнес-показатели" команды

## Пример

In [10]:
class GeneticSolver:
    def __init__(self, func, steps=100, genes=10, cut_genes=2, chrom=5):
        '''
        :param func: Optimizing function
        :param steps: Number of iterations
        :param genes: number of gene
        :param cut_genes: number of gene to replace
        :param chrom: number of chromosomes
        '''
        self.f = func
        self.steps = steps
        self.genes = genes
        self.cut_genes = cut_genes
        self.chrom = chrom
        self.X = []
        self.init_genes()
        self.solve()

    def fit(self, gene):
        return 1.0 / (1 + self.f(gene))

    def sort(self):
        res = self.X
        for m in range(self.genes - 1):
            for i in range(self.genes - m - 1):
                if self.fit(res[i]) < self.fit(res[i + 1]):
                    c = res[i]
                    res[i] = res[i + 1]
                    res[i + 1] = c
        self.X = res

    def cross(self, gene_a, gene_b):
        res = []
        p = self.fit(gene_a) / (self.fit(gene_a) + self.fit(gene_b))
        for n in range(self.chrom):
            if np.random.uniform(0, 1) < p:
                res.append(gene_a[n])
            else:
                res.append(gene_b[n])
        return res

    def generate_gene(self):
        return [np.random.uniform(-1, 1) for _ in range(self.chrom)]

    def init_genes(self):
        [self.X.append(self.generate_gene()) for _ in range(self.genes)]

    def solve(self):
        for l in range(self.steps):
            self.sort()
            for m in range(1, self.genes - self.cut_genes):
                self.X[m] = self.cross(self.X[m],
                                       self.X[np.random.randint(0, self.genes)])
            for m in range(self.genes - self.cut_genes, self.genes):
                self.X[m] = self.generate_gene()


In [12]:
def f(x):
    res = 0
    for xn in x:
        res += xn * xn * (1 + np.abs(np.sin(100 * xn)))
    return res

G = GeneticSolver(f)

In [13]:
G.X[0]

[0.006580087379671973,
 0.029151313750332974,
 -0.0006567536970703003,
 -0.103576665357213,
 0.0630326291707568]

In [5]:
f(G.X[0])

0.046773614361303596